In [1]:
import os
import pickle
import pandas as pd
import re
import numpy as np


# Merge the data

In [2]:
# Directory containing the pickle files
#DATA_DIR =  r"Z:\neuronies\single_neurons\1_Subprojects\Neurons_As_DNNs\3_Processed_Data\March2025_heart\biTE_stimulation\Full_dataset\Full_files"
#OUTPUT_DIR = r"Z:\neuronies\single_neurons\1_Subprojects\Neurons_As_DNNs\3_Processed_Data\March2025_heart\biTE_stimulation\Full_dataset\CSVs"
DATA_DIR = '/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Neurons_As_DNNs/3_Processed_Data/March2025_heart/biTE_stimulation/Full_dataset/Full_files'
OUTPUT_DIR = '/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Neurons_As_DNNs/3_Processed_Data/March2025_heart/biTE_stimulation/Full_dataset/CSVs'

DATA_KEYS = ["validated_results"]

In [3]:
# List to store all dataframes
all_data = []

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Dictionary to store all extracted data
data_dict = {key: [] for key in DATA_KEYS}

# Loop through all pickle files in the directory
for file in os.listdir(DATA_DIR):
    if file.endswith(".pkl"):
        file_path = os.path.join(DATA_DIR, file)

        try:
            with open(file_path, "rb") as f:
                data = pickle.load(f)

            for key in DATA_KEYS:
                if key in data:
                    df_data = pd.DataFrame(data[key]) # Convert structured array to DataFrame
                    df = df_data[["source_electrode", "target_electrode",
                      "source_unit_id", "target_unit_id",
                      "lag", "validation", "mTE",
                      "syn probability", "latency_extremum"]
                    ]

                    electrodes_sources = []
                    electrodes_targets = []
                    for _, row in df.iterrows():
                        electrodes_sources.append(data["UNIT_TO_EL"][row["source_unit_id"]])
                        electrodes_targets.append(data["UNIT_TO_EL"][row["target_unit_id"]])

                    df["electrodes_source_unit"] = electrodes_sources
                    df["electrodes_target_unit"] = electrodes_targets
                    df["filename"] = file
                    data_dict[key].append(df)
                
                else:
                    print(f"Warning: key {key} was not in file {file}.")

        except Exception as e:
            print(f"Error processing {file}: {e}")


# Save each dataset as a separate CSV
for key in DATA_KEYS:
    if data_dict[key]:
        merged_df = pd.concat(data_dict[key], ignore_index=True)
        output_file = os.path.join(OUTPUT_DIR, f"{key}_full_data_w_speed_and_firing.pkl")
        #merged_df.to_csv(output_file, index=False)
        with open(output_file, "wb") as f: 
            pickle.dump(merged_df, f)
        print(f"Saved {key} data to {output_file}")
    else:
        print(f"No valid {key} data found.")

Saved validated_results data to /itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Neurons_As_DNNs/3_Processed_Data/March2025_heart/biTE_stimulation/Full_dataset/CSVs/validated_results_full_data_w_speed_and_firing.pkl


# Add information to the data

In [4]:
#data = pd.read_csv(r"Z:\neuronies\single_neurons\1_Subprojects\Neurons_As_DNNs\3_Processed_Data\March2025_heart\biTE_stimulation\Full_dataset\CSVs\validated_results_full_data.csv")
data = np.load('/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Neurons_As_DNNs/3_Processed_Data/March2025_heart/biTE_stimulation/Full_dataset/CSVs/validated_results_full_data_w_speed_and_firing.pkl', allow_pickle=True)
data = pd.DataFrame(data)

In [8]:
# Function to parse the paste.txt content and create a lookup dictionary
def parse_experiment_info(text_content):
    # Split the content by sections (each experiment)
    sections = re.split(r'-{20,}', text_content)

    # Dictionary to store experiment information by filename
    experiment_lookup = {}

    # Process each section
    current_experiment = None
    current_frequency = None
    current_delay = None
    current_chip_id = None
    current_network = None
    current_div = None
    current_repetition = None
    current_status = None

    for section in sections:
        if not section.strip():
            continue

        # Extract experiment details
        exp_match = re.search(r'Experiment:\s*([^\n]+)', section)
        if exp_match:
            current_experiment = exp_match.group(1).strip()

        freq_match = re.search(r'Stimulation frequency:\s*([^\n]+)', section)
        if freq_match:
            current_frequency = freq_match.group(1).strip()

        delay_match = re.search(r'Delay:\s*([^\n]+)', section)
        if delay_match:
            current_delay = delay_match.group(1).strip()

        chip_match = re.search(r'Chip ID:\s*([^\n]+)', section)
        if chip_match:
            current_chip_id = chip_match.group(1).strip()

        network_match = re.search(r'Network:\s*([^\n]+)', section)
        if network_match:
            current_network = network_match.group(1).strip()

        div_match = re.search(r'DIV:\s*([^\n]+)', section)
        if div_match:
            current_div = div_match.group(1).strip()

        rep_match = re.search(r'Repetition:\s*([^\n]+)', section)
        if rep_match:
            current_repetition = rep_match.group(1).strip()

        # Process status and filename pairs
        status_file_pairs = re.findall(r'Status:\s*(before|after|after_2)\s*\n(ID\d+_\d+_DIV\d+_DATE\d+_\d+_[^\.]+\.raw_processed_info_metrics\.pkl)', section, re.IGNORECASE)

        for status, filename in status_file_pairs:
            experiment_lookup[filename] = {
                'Experiment': current_experiment,
                'Stimulation_frequency': current_frequency,
                'Delay': current_delay,
                'Chip_ID': current_chip_id,
                'Network': current_network,
                'DIV': current_div,
                'Repetition': current_repetition,
                'Status': status.lower()
            }

    return experiment_lookup


In [9]:
# Read info text document
#with open(r"Z:\neuronies\single_neurons\1_Subprojects\Neurons_As_DNNs\3_Processed_Data\March2025_heart\biTE_stimulation\Full_dataset\info_stimulation_experiments.txt", 'r') as file:
with open('/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Neurons_As_DNNs/3_Processed_Data/March2025_heart/biTE_stimulation/Full_dataset/info_stimulation_experiments.txt', 'r') as file:
    # Read the content of the file
    text_content = file.read()

# Create lookup dictionary from the paste.txt content
experiment_lookup = parse_experiment_info(text_content)
print(experiment_lookup)

{'ID2184_7_DIV32_DATE20250415_1156_spontaneous_Freq_variation_try_2.raw_processed_info_metrics.pkl': {'Experiment': 'Frequency_Variation', 'Stimulation_frequency': '5_Hz', 'Delay': '0_ms', 'Chip_ID': 'ID2184', 'Network': '7', 'DIV': '32', 'Repetition': '1', 'Status': 'before'}, 'ID2184_7_DIV32_DATE20250415_1207_spontaneous_Freq_variation_try_2.raw_processed_info_metrics.pkl': {'Experiment': 'Frequency_Variation', 'Stimulation_frequency': '5_Hz', 'Delay': '0_ms', 'Chip_ID': 'ID2184', 'Network': '7', 'DIV': '32', 'Repetition': '1', 'Status': 'after'}, 'ID2184_7_DIV32_DATE20250415_1207_spontaneous_Freq_variation_try_2_0.raw_processed_info_metrics.pkl': {'Experiment': 'Frequency_Variation', 'Stimulation_frequency': '5_Hz', 'Delay': '0_ms', 'Chip_ID': 'ID2184', 'Network': '7', 'DIV': '32', 'Repetition': '1', 'Status': 'after_2'}, 'ID2184_7_DIV32_DATE20250415_1225_spontaneous_Freq_variation_try_2.raw_processed_info_metrics.pkl': {'Experiment': 'Frequency_Variation', 'Stimulation_frequency': 

In [10]:
# Function to add experiment information to the dataframe
def add_experiment_info(row, lookup_dict):
    filename = row['filename']

    # Get info from lookup dictionary
    try:
        if filename in lookup_dict:
            info = lookup_dict[filename]
            return pd.Series([
                info.get('Experiment', None),
                info.get('Stimulation_frequency', None),
                info.get('Delay', None),
                info.get('Chip_ID', None),
                info.get('Network', None),
                info.get('DIV', None),
                info.get('Repetition', None),
                info.get('Status', None)
            ])

    except Exception as e:
        print(f"Error adding filename {filename}")

# Add the new columns to your dataframe
new_cols = ['Experiment', 'Stimulation_frequency', 'Delay', 'Chip_ID', 'Network', 'DIV', 'Repetition', "Status"]
data[new_cols] = data.apply(lambda row: add_experiment_info(row, experiment_lookup), axis=1)

# Display the updated dataframe
print(data[['filename'] + new_cols].head())



                                            filename           Experiment  \
0  ID2184_1_DIV35_DATE20250418_1518_spontaneous_F...  Frequency_Variation   
1  ID2184_1_DIV35_DATE20250418_1518_spontaneous_F...  Frequency_Variation   
2  ID2184_1_DIV35_DATE20250418_1518_spontaneous_F...  Frequency_Variation   
3  ID2184_1_DIV35_DATE20250418_1518_spontaneous_F...  Frequency_Variation   
4  ID2184_1_DIV35_DATE20250418_1518_spontaneous_F...  Frequency_Variation   

  Stimulation_frequency Delay Chip_ID Network DIV Repetition   Status  
0                 10_Hz  5_ms  ID2184       1  35          1  after_2  
1                 10_Hz  5_ms  ID2184       1  35          1  after_2  
2                 10_Hz  5_ms  ID2184       1  35          1  after_2  
3                 10_Hz  5_ms  ID2184       1  35          1  after_2  
4                 10_Hz  5_ms  ID2184       1  35          1  after_2  


In [11]:
data

,source_electrode,target_electrode,source_unit_id,target_unit_id,lag,validation,mTE,syn probability,latency_extremum,electrodes_source_unit,electrodes_target_unit,filename,Experiment,Stimulation_frequency,Delay,Chip_ID,Network,DIV,Repetition,Status
0,17884,13264,92,60,10.0,bad,0.000047,NaN,"[[0, 474.95, 0.0, input], [1, 1251.30000000000...","[19864, 18984, 18104, 15464, 17224, 19424, 150...","[11718, 11057, 13264, 12157, 11715, 11277, 112...",ID2184_1_DIV35_DATE20250418_1518_spontaneous_F...,Frequency_Variation,10_Hz,5_ms,ID2184,1,35,1,after_2
1,10833,21840,104,99,10.0,bad,0.000053,NaN,"[[0, 99.35, 0.0, input], [1, 467.2, 0.0, input...","[9954, 10834, 9735, 11934, 7092, 11933, 10833,...","[20740, 20519, 20521, 21182, 21399, 22940, 218...",ID2184_1_DIV35_DATE20250418_1518_spontaneous_F...,Frequency_Variation,10_Hz,5_ms,ID2184,1,35,1,after_2
2,11055,21840,57,99,7.0,bad,0.000046,NaN,"[[0, 1100.95, 0.0, input], [1, 2815.15, 0.0, i...","[9954, 10834, 9735, 11935, 11934, 11715, 11276...","[20740, 20519, 20521, 21182, 21399, 22940, 218...",ID2184_1_DIV35_DATE20250418_1518_spontaneous_F...,Frequency_Variation,10_Hz,5_ms,ID2184,1,35,1,after_2
3,11057,7769,87,1,5.0,bad,0.000055,NaN,"[[0, 89.5, 0.0, input], [1, 466.55, 0.0, input...","[11718, 9954, 11057, 11058, 10834, 10177, 9735...","[15028, 11069, 10409, 13929, 9529, 14148, 1480...",ID2184_1_DIV35_DATE20250418_1518_spontaneous_F...,Frequency_Variation,10_Hz,5_ms,ID2184,1,35,1,after_2
4,2478,9296,2,52,3.0,bad,0.000057,NaN,"[[0, 489.4, 0.0, input], [1, 1091.6, 0.0, inpu...","[11057, 11277, 11276, 5555, 3136, 6654, 2477, ...","[10178, 9954, 11057, 10619, 10177, 9735, 12157...",ID2184_1_DIV35_DATE20250418_1518_spontaneous_F...,Frequency_Variation,10_Hz,5_ms,ID2184,1,35,1,after_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26315,9820,14000,54,26,2.0,bad,0.000062,NaN,"[[0, 20.75, 0.0, input], [1, 71.2, 0.0, input]...","[11800, 10701, 10040, 7400, 8061, 6739, 9160, ...","[11800, 14655, 21274, 21050, 15754, 20392, 192...",ID2184_4_DIV35_DATE20250418_1304_spontaneous_F...,Frequency_Variation,10_Hz,0_ms,ID2184,4,35,1,before
26316,1901,14000,65,26,6.0,bad,0.000058,NaN,"[[0, 1590.1000000000001, 0.0, input], [1, 3222...","[11800, 7400, 2558, 5640, 3219, 1682, 1678, 27...","[11800, 14655, 21274, 21050, 15754, 20392, 192...",ID2184_4_DIV35_DATE20250418_1304_spontaneous_F...,Frequency_Variation,10_Hz,0_ms,ID2184,4,35,1,before
26317,1686,14226,87,10,7.0,bad,0.000024,NaN,"[[0, 90.25, 0.0, input], [1, 126.2, 0.0, input...","[3444, 2566, 5425, 5206, 1688, 1686, 1248, 432...","[14226, 21050, 13785, 20392, 19293, 19954, 155...",ID2184_4_DIV35_DATE20250418_1304_spontaneous_F...,Frequency_Variation,10_Hz,0_ms,ID2184,4,35,1,before
26318,2776,2788,46,15,2.0,bad,0.000048,NaN,"[[0, 548.3, 0.0, input], [1, 1218.6, 0.0, inpu...","[2558, 4318, 4756, 3219, 1678, 2121, 4100, 431...","[11147, 7848, 9608, 4547, 10268, 7627, 9168, 6...",ID2184_4_DIV35_DATE20250418_1304_spontaneous_F...,Frequency_Variation,10_Hz,0_ms,ID2184,4,35,1,before


In [13]:
#data.to_csv(r"Z:\neuronies\single_neurons\1_Subprojects\Neurons_As_DNNs\3_Processed_Data\March2025_heart\biTE_stimulation\Full_dataset\CSVs\validated_results_full_data.csv", index=False)
# Save the updated dataframe to a new pickle file
data.to_pickle('/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Neurons_As_DNNs/3_Processed_Data/March2025_heart/biTE_stimulation/Full_dataset/CSVs/validated_results_full_data_w_speed_and_firing.pkl')